# Machine Learning HW1

## PM2.5 Prediction

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('ml2020spring-hw1/train.csv', encoding = 'big5')
data.head(18)

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [3]:
data1 = data.iloc[:,3:]
data1[data1 == 'NR'] = 0
data1.shape

(4320, 24)

In [4]:
raw_data = data1.to_numpy()

### Extract Feature 1

将 $ 4320 * 18$ 的数据转为 $12 * 20 * 24 * 18 $ 的数据，12个月，每月20天，每天24小时，每小时18个特征。

必须分开12个月计算，每个月的时间不连续。

In [5]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:,day * 24 : (day+1) * 24] = raw_data[18 * (month * 20 + day) : 18 * (month * 20 + day + 1),:]
    month_data[month] = sample

In [6]:
# 每个月数据均为 18 * 480
month_data[11]

array([[ 23.  ,  23.  ,  23.  , ...,  13.  ,  13.  ,  13.  ],
       [  1.6 ,   1.7 ,   1.7 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.22,   0.2 ,   0.18, ...,   0.51,   0.57,   0.56],
       ...,
       [ 93.  ,  50.  ,  99.  , ..., 118.  , 100.  , 105.  ],
       [  1.8 ,   2.1 ,   3.2 , ...,   1.5 ,   2.  ,   2.  ],
       [  1.3 ,   0.9 ,   1.  , ...,   1.6 ,   1.8 ,   2.  ]])

### Extract Feature 2
每个月连续时间480小时，每9个小时产生一组data，每月共471组连续数据。

In [7]:
x_data = np.empty([471 * 12, 18 * 9], dtype = float)
y_data = np.empty([471 * 12, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x_data[month * 471 + day * 24 + hour,:] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            y_data[month * 471 + day * 24 + hour,:] = month_data[month][9,day * 24 + hour + 9]
print(x_data.shape)
print(y_data.shape)

(5652, 162)
(5652, 1)


### Normalization

In [8]:
x_mean = np.mean(x_data, axis = 0)
x_std = np.std(x_data, axis = 0)
x_mean.shape,x_std.shape

((162,), (162,))

In [99]:
for i in range(len(x_data)):
    for j in range(len(x_data[i])):
        if x_std[j]:
            x_data[i][j] = (x_data[i][j] - x_mean[j]) / x_std[j]
x_data,y_data

(array([[-4.26244109, -4.2619117 , -4.26059692, ..., -6.36737333,
         -6.1489991 , -6.08935727],
        [-4.26223714, -4.26170811, -4.26029228, ..., -4.41557358,
         -4.99414807, -5.43744613],
        [-4.26182923, -4.26140272, -4.26019073, ..., -5.86135117,
         -5.64375177, -6.30666098],
        ...,
        [-4.26264504, -4.26211529, -4.26100312, ..., -6.15050669,
         -6.00464272, -5.94448813],
        [-4.26091144, -4.26048656, -4.25927679, ..., -4.84930686,
         -4.48890074, -5.5098807 ],
        [-4.26264504, -4.26221708, -4.26100312, ..., -6.80110661,
         -6.72642462, -6.74126841]]), array([[30.],
        [41.],
        [44.],
        ...,
        [17.],
        [24.],
        [29.]]))

### Shuffle the set

此数据集有明显的时间连续性

In [68]:
import random
index = [i for i in range(len(x_data))]
random.shuffle(index)
x_data = x_data[index]

### Split Validation Set

In [70]:
import math
x_train, y_train = x_data[:math.floor(len(x_data) * 0.8),:], y_data[:math.floor(len(x_data) * 0.8),:]
x_val, y_val = x_data[math.floor(len(x_data) * 0.8):,:], y_data[math.floor(len(x_data) * 0.8):,:]

### Linear Regression

### Adagrad

In [100]:
iteration = 3000
learning_rate = 100
eps = 1e-7
rows = int(x_train.shape[0])
params = int(x_train.shape[1])

w = np.matrix(np.zeros([params+1, 1]))
x = np.matrix(np.concatenate((np.ones([rows, 1]),x_train), axis = 1).astype(float))
y = np.matrix(y_train)

adgrad = np.matrix(np.zeros([params+1, 1]))
for item in range(iteration):
    # loss function is MSE
    loss = np.sum(np.power((x * w - y), 2)) / (rows * 2)
    gradient = x.T * (x * w - y) / rows
    adgrad += np.power(gradient, 2)
    w = w - np.multiply(learning_rate / (np.sqrt(adgrad) + eps) , gradient)
    if (item+1) % 100 == 0:
        print(f'No.{item+1} loss is {loss}')
w

[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
No.100 loss is 157.3468091694279
No.200 loss is 152.1601461781137
No.300 loss is 149.1810719336844
No.400 loss is 147.4314397137706
No.500 loss is 146.38377865260102
No.600 loss is 145.7424973073254
No.700 loss is 145.33917164605842
No.800 loss is 145.0767904890207
No.900 loss is 144.89894501284178
No.1000 loss is 144.7725194758051
No.1100 loss is 144.67787256082963
No.1200 loss is 144.60322489226797
No.1300 loss is 144.54142692474096
No.1400 loss is 144.48808296773777
No.1500 loss is 144.44045302883313
No.1600 loss is 144.39680400503755
No.1700 loss is 144.35602251680538
No.1800 loss is 144.3173814065214
No.1900 loss is 144.2803973316256
No.2000 loss is 144.24474290290888
No.2100 loss is 144.21019183011663
No.2200 loss is 144.17658426248886
No.2300 loss is 144.14380462540268
No.2400 loss is 144.1117672779271
No.2500 loss is 144.0804071211888
No.2600 loss is 144.04967337602676
No.2700 loss is 144.01952541183684
No.2800 loss is 143.9899299

matrix([[ 1.52674501e-02],
        [-1.52714488e-02],
        [-1.52711393e-02],
        [-1.52712133e-02],
        [-1.52714028e-02],
        [-1.52714803e-02],
        [-1.52719056e-02],
        [-1.52720923e-02],
        [-1.52724285e-02],
        [-1.52722416e-02],
        [ 4.25222159e-04],
        [ 2.04756781e-04],
        [ 1.91023183e-04],
        [ 3.49362404e-04],
        [ 2.03627366e-04],
        [ 4.82137766e-04],
        [ 8.39925696e-05],
        [ 2.80117844e-04],
        [ 3.67740132e-04],
        [ 2.47538666e-04],
        [-1.11972727e-03],
        [-1.08147325e-03],
        [-5.83951669e-03],
        [-1.68752578e-03],
        [ 4.68471210e-03],
        [ 9.63163814e-04],
        [ 9.18219311e-04],
        [ 6.60646308e-04],
        [ 1.93888945e-04],
        [ 1.88586903e-04],
        [ 1.01558342e-04],
        [ 1.03733105e-04],
        [ 1.67116202e-04],
        [-3.96524246e-05],
        [ 2.04296201e-04],
        [ 3.72623376e-05],
        [ 2.47176317e-04],
 

### Predict PM2.5

In [96]:
test_data = pd.read_csv('ml2020spring-hw1/test.csv', encoding = 'big5', header = None)
test_data[test_data == 'NR'] = 0
test_data = test_data.iloc[:,2:]

test_data = test_data.to_numpy()
x_test = np.empty([240, 18 * 9])

for idx in range(240):
    x_test[idx,:] = test_data[idx * 18:(idx + 1) * 18,:].reshape(1,-1)

test_mean = np.mean(x_test, axis = 0)
test_std = np.mean(x_test, axis = 0)
for i in range(int(x_test.shape[0])):
    for j in range(int(x_test.shape[1])):
        x_test[i][j] = (x_test[i][j] - test_mean[j]) / test_std[j]

x_test = np.concatenate((np.ones([int(x_test.shape[0]), 1]), x_test), axis = 1 ).astype(float)
x_test = np.matrix(x_test)
x_test.shape


(240, 163)

In [102]:
y_pre = x_test * w
for i in range(len(y_pre)):
    print(f"id_{i+1} : {y_pre[i]}")

id_1 : [[0.57070218]]
id_2 : [[0.65235473]]
id_3 : [[-0.30701628]]
id_4 : [[0.40241385]]
id_5 : [[0.26493919]]
id_6 : [[0.37876118]]
id_7 : [[0.26751866]]
id_8 : [[0.25872025]]
id_9 : [[0.08350114]]
id_10 : [[-0.54046536]]
id_11 : [[0.36352639]]
id_12 : [[0.00556425]]
id_13 : [[-0.15485916]]
id_14 : [[-0.31624701]]
id_15 : [[0.27171705]]
id_16 : [[0.57299236]]
id_17 : [[-0.22793238]]
id_18 : [[-0.73573344]]
id_19 : [[-3.0895214]]
id_20 : [[0.52877332]]
id_21 : [[0.3065869]]
id_22 : [[-0.41542618]]
id_23 : [[0.23809431]]
id_24 : [[-0.01459011]]
id_25 : [[0.7680247]]
id_26 : [[-0.03546169]]
id_27 : [[0.5654344]]
id_28 : [[-0.78118857]]
id_29 : [[0.47692801]]
id_30 : [[-0.79470123]]
id_31 : [[-0.27446418]]
id_32 : [[0.39926989]]
id_33 : [[0.69793198]]
id_34 : [[0.06732249]]
id_35 : [[-0.39245098]]
id_36 : [[-0.1400854]]
id_37 : [[0.33426404]]
id_38 : [[0.01584088]]
id_39 : [[-0.88209256]]
id_40 : [[-0.21810047]]
id_41 : [[-0.09417422]]
id_42 : [[0.30681507]]
id_43 : [[-0.11380245]]
id_44 

**输出和理想结果差距较大，考虑将在train时将label y的值同样归一化。**